In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [12]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [38]:
config = tf.ConfigProto(intra_op_parallelism_threads=4, 
                        inter_op_parallelism_threads=4, 
                        allow_soft_placement=True)

session = tf.Session(config=config)

In [3]:
data = pd.read_csv('./data/hour_preprocessed.csv')

In [4]:
X = data.iloc[:,0:-1]
y = data.iloc[:,-1]

In [6]:
test_size=0.33
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=43)

y_train = y_train.values.flatten()
y_test = y_test.values.flatten()

In [9]:
SIZE_INPUT = 19
SIZE_OUTPUT = 1

SIZE_HIDDEN = 43

In [10]:
def build_model():
  model = keras.Sequential([
    layers.Dense(SIZE_INPUT, activation=tf.nn.relu, input_shape=[len(X_train.keys())]),
    
    layers.Dense(SIZE_HIDDEN, activation=tf.nn.relu),
    layers.Dense(SIZE_HIDDEN, activation=tf.nn.relu),
    layers.Dense(SIZE_HIDDEN, activation=tf.nn.relu),
    layers.Dense(SIZE_HIDDEN, activation=tf.nn.relu),
    layers.Dense(SIZE_HIDDEN, activation=tf.nn.relu),
    layers.Dense(SIZE_HIDDEN, activation=tf.nn.relu),
    layers.Dense(SIZE_HIDDEN, activation=tf.nn.relu),
    layers.Dense(SIZE_HIDDEN, activation=tf.nn.relu),
    
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mean_squared_error',
                optimizer=optimizer,
                metrics=['mean_absolute_error', 'mean_squared_error'])
  return model

In [13]:
model = build_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 19)                380       
_________________________________________________________________
dense_1 (Dense)              (None, 43)                860       
_________________________________________________________________
dense_2 (Dense)              (None, 43)                1892      
_________________________________________________________________
dense_3 (Dense)              (None, 43)                1892      
_________________________________________________________________
dense_4 (Dense)              (None, 43)                1892      
_________________________________________________________________
dense_5 (Dense)              (None, 43)                1892      
_________________________________________________________________
dense_6 (Dense)              (None, 43)                1892      
__________

In [20]:
from utils.printdot import PrintDot

EPOCHS = 1000
# The patience parameter is the amount of epochs to check for improvement
early_stop = keras.callbacks.EarlyStopping(monitor='mean_squared_error', patience=10)

history = model.fit(
  X_train, 
  y_train,
  epochs=EPOCHS, 
  validation_split = 0.2, 
  verbose=0,
  callbacks=[early_stop, PrintDot()])


....................................................................................................
....................................................................................................
..................................

In [21]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

,val_loss,val_mean_absolute_error,val_mean_squared_error,loss,mean_absolute_error,mean_squared_error,epoch
229,3387.197470,35.938905,3387.197470,1287.723613,22.627280,1287.723613,229
230,3583.911405,37.970626,3583.911405,1283.962439,22.762362,1283.962439,230
231,2351.273365,29.150850,2351.273365,1294.638599,22.721704,1294.638599,231
232,3950.730103,39.408807,3950.730103,1284.217564,22.663050,1284.217564,232
233,2304.678022,29.447053,2304.678022,1277.763608,22.595957,1277.763608,233


In [59]:
y_pred = model.predict(X_test)
y_test = y_test.reshape(-1,1)

In [62]:
y_pred.shape == y_test.shape

True

In [63]:
# Compute R-square score
total_error = tf.reduce_sum(tf.square(tf.subtract(y_test, tf.reduce_mean(y_test))))
unexplained_error = tf.reduce_sum(tf.square(tf.subtract(y_test, y_pred)))
R_squared = tf.subtract(tf.cast(1.0, tf.int64), tf.div(unexplained_error, total_error))

In [66]:
R_squared.eval(session=session)

1

In [70]:
r2 = 1.0 - (unexplained_error.eval(session=session) / total_error.eval(session=session))

In [71]:
print("R-squared (Pred)::{}".format(r2))

R-squared (Pred)::0.9340234430634813
